In this notebook we design a multi-modal model using GraphSAGE.

In [65]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [66]:
cd drive/MyDrive/Colab\ Notebooks

[Errno 2] No such file or directory: 'drive/MyDrive/Colab Notebooks'
/content/drive/MyDrive/Colab Notebooks


In [67]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import keras
from keras.layers import Dense
from keras.models import Sequential
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torchtext
import torch
from torch.nn import Linear
import itertools
import nltk
import spacy
import random
from tqdm import tqdm
import time
from IPython.display import Javascript
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, Normalizer
from sklearn.metrics import roc_auc_score, confusion_matrix, roc_curve, classification_report

In [68]:
import numpy as np
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
import math
from collections import Counter
import random
import csv
import os
import sys

In [69]:
econ_df = pd.read_csv('./datasets/la_0812.csv')
econ_df.tail(10)

,Own children of the householder 6 to 17 years,YEAR OF ENTRY Foreign born,GROSS RENT Occupied units paying rent Median (dollars),Sex ratio (males per 100 females),"Educational services, and health care and social assistance",relation_max_fee,White,ROOM Median rooms,Hispanic or Latino (of any race),Mean household income (dollars),...,HOUSING TENURE Renter-occupied,"Arts, entertainment, and recreation, and accommodation and food services",SELECTED MONTHLY OWNER COSTS (SMOC) Housing units without a mortgage Median (dollars),SELECTED MONTHLY OWNER COSTS (SMOC) Housing units with a mortgage Median (dollars),Own children of the householder under 6 years,label,INTPTLONG,INTPTLAT,geoid,Median age (years)
1727,1481,2851,1368.000000,121.400000,0.100000,1861.0,0.690000,3.90000,0.800000,57555.00000,...,0.780000,0.250000,644.000000,2156.000000,680,0,-118.535884,34.385597,6037920336,27.300000
1728,1567,2595,1694.000000,117.200000,0.180000,1861.0,0.660000,4.00000,0.690000,76192.00000,...,0.710000,0.140000,706.000000,2281.000000,481,0,-118.512874,34.373662,6037920337,28.800000
1729,1156,1460,2025.000000,83.000000,0.220000,2412.0,0.680000,7.40000,0.170000,176276.00000,...,0.100000,0.080000,1233.000000,3449.000000,563,0,-118.596430,34.386084,6037920338,38.800000
1730,1784,1903,2412.000000,98.900000,0.250000,2810.0,0.630000,6.70000,0.120000,184362.00000,...,0.250000,0.070000,1472.000000,3592.000000,469,0,-118.612172,34.415357,6037920339,36.500000
1731,17,76,1750.000000,209.300000,0.300000,2025.0,0.690000,5.20000,0.210000,92146.00000,...,0.120000,0.050000,494.000000,1375.000000,0,0,-117.866141,34.280333,6037930301,45.100000
1732,76,80,1062.000000,110.600000,0.290000,2209.0,0.290000,2.60000,0.190000,67300.00000,...,1.000000,0.000000,607.726253,2403.218731,44,0,-118.288839,33.773247,6037980015,31.700000
1733,9,39,3250.000000,54.100000,0.390000,3500.0,0.830000,5.70000,0.000000,221046.00000,...,0.170000,0.050000,850.000000,3944.000000,14,0,-118.534363,34.092567,6037980019,56.400000
1734,4,0,1538.682558,200.000000,0.000000,1992.0,0.330000,4.66761,0.670000,88101.58131,...,0.000000,0.000000,607.726253,2403.218731,0,0,-118.381334,34.270047,6037980021,52.300000
1735,5,157,1900.000000,4537.500000,0.170000,1369.0,0.550000,4.10000,0.330000,97925.00000,...,1.000000,0.000000,607.726253,2403.218731,19,0,-118.258539,33.739090,6037980031,44.100000
1736,0,0,1538.682558,131.931509,0.202386,1864.0,0.516653,4.66761,0.492621,88101.58131,...,0.546156,0.112778,607.726253,2403.218731,0,1,-117.245574,32.685362,6073009901,36.828514


In [70]:
econ_df.columns

Index(['Own children of the householder 6 to 17 years',
       'YEAR OF ENTRY Foreign born',
       'GROSS RENT Occupied units paying rent Median (dollars)',
       'Sex ratio (males per 100 females)',
       'Educational services, and health care and social assistance',
       'relation_max_fee', 'White', 'ROOM Median rooms',
       'Hispanic or Latino (of any race)', 'Mean household income (dollars)',
       'VALUE Median (dollars)', 'Mean travel time to work (minutes)',
       'Total population', 'Rental vacancy rate', 'relation_min_fee',
       'HOUSING TENURE Renter-occupied',
       'Arts, entertainment, and recreation, and accommodation and food services',
       'SELECTED MONTHLY OWNER COSTS (SMOC) Housing units without a mortgage Median (dollars)',
       'SELECTED MONTHLY OWNER COSTS (SMOC) Housing units with a mortgage Median (dollars)',
       'Own children of the householder under 6 years', 'label', 'INTPTLONG',
       'INTPTLAT', 'geoid', 'Median age (years)'],
      dtyp

In [71]:
ids = econ_df['geoid']
labels = econ_df['label']
lats = econ_df['INTPTLAT']
longs = econ_df['INTPTLONG']

In [72]:
object_df = pd.read_csv('./datasets/la_object_detection.csv')
object_df

,tract_id,house_num,shape_area,shape_length,target
0,6037408134,690,2.211666e+06,151661.376601,0
1,6037408135,895,3.043843e+06,203541.457136,0
2,6037408136,1114,3.393638e+06,241308.785750,0
3,6037408137,586,2.764442e+06,156781.092347,0
4,6037408138,1112,2.850623e+06,220997.776154,0
...,...,...,...,...,...
1647,6073003115,1407,4.217873e+06,317006.934760,0
1648,6073003201,944,3.683951e+06,233576.994501,0
1649,6073004600,308,8.072192e+05,61696.222360,0
1650,6073007002,1459,4.213008e+06,309635.972875,0


In [73]:
shortest_df = pd.read_csv('./datasets/la_shortest_paths.csv')
shortest_df

,id,hospital_1,hospital_2,subway_1,subway_2,school_1,school_2
0,6.037101e+09,4.0,8.0,10,10,1,1
1,6.037101e+09,5.0,9.0,11,11,1,1
2,6.037101e+09,4.0,8.0,10,10,1,1
3,6.037101e+09,3.0,7.0,9,9,1,1
4,6.037102e+09,6.0,7.0,9,9,1,1
...,...,...,...,...,...,...,...
1709,6.037930e+09,2.0,3.0,9,18,18,19
1710,6.037980e+09,4.0,5.0,13,13,1,1
1711,6.037980e+09,4.0,6.0,7,8,2,2
1712,6.037980e+09,6.0,10.0,16,16,1,1


In [74]:
combined_df = pd.merge(econ_df, object_df, left_on='geoid', right_on='tract_id')
combined_df = pd.merge(combined_df, shortest_df, left_on='geoid', right_on='id')
combined_df

,Own children of the householder 6 to 17 years,YEAR OF ENTRY Foreign born,GROSS RENT Occupied units paying rent Median (dollars),Sex ratio (males per 100 females),"Educational services, and health care and social assistance",relation_max_fee,White,ROOM Median rooms,Hispanic or Latino (of any race),Mean household income (dollars),...,shape_area,shape_length,target,id,hospital_1,hospital_2,subway_1,subway_2,school_1,school_2
0,489,1449,1609.0,95.8,0.25,2120.0,0.77,4.5,0.32,69041.0,...,3.120899e+06,237208.582607,0,6.037101e+09,4.0,8.0,10,10,1,1
1,351,846,2120.0,90.1,0.25,2016.0,0.88,5.4,0.04,118157.0,...,3.809366e+06,286851.325957,0,6.037101e+09,5.0,9.0,11,11,1,1
2,804,3024,1318.0,124.1,0.24,2120.0,0.79,3.6,0.43,50062.0,...,2.277990e+06,129988.741573,0,6.037101e+09,4.0,8.0,10,10,1,1
3,538,1686,1198.0,84.7,0.22,2120.0,0.71,4.0,0.38,56903.0,...,1.694219e+06,123837.662402,0,6.037101e+09,3.0,7.0,9,9,1,1
4,181,764,2111.0,84.5,0.26,2063.0,0.86,5.1,0.27,92459.0,...,1.841664e+06,129346.548908,0,6.037102e+09,6.0,7.0,9,9,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1624,844,1256,1718.0,95.7,0.26,2046.0,0.72,5.3,0.22,103189.0,...,6.408532e+06,374804.050968,0,6.037920e+09,2.0,10.0,12,12,3,3
1625,1481,2851,1368.0,121.4,0.10,1861.0,0.69,3.9,0.80,57555.0,...,4.439727e+06,275777.142775,0,6.037920e+09,3.0,11.0,13,13,4,4
1626,1567,2595,1694.0,117.2,0.18,1861.0,0.66,4.0,0.69,76192.0,...,2.982897e+06,193121.095700,0,6.037920e+09,3.0,12.0,14,14,5,5
1627,1156,1460,2025.0,83.0,0.22,2412.0,0.68,7.4,0.17,176276.0,...,5.064067e+06,348448.703378,0,6.037920e+09,2.0,9.0,12,12,3,3


In [75]:
combined_df[['geoid', 'id']]

,geoid,id
0,6037101110,6.037101e+09
1,6037101122,6.037101e+09
2,6037101210,6.037101e+09
3,6037101220,6.037101e+09
4,6037102103,6.037102e+09
...,...,...
1624,6037920334,6.037920e+09
1625,6037920336,6.037920e+09
1626,6037920337,6.037920e+09
1627,6037920338,6.037920e+09


In [76]:
combined_df.columns

Index(['Own children of the householder 6 to 17 years',
       'YEAR OF ENTRY Foreign born',
       'GROSS RENT Occupied units paying rent Median (dollars)',
       'Sex ratio (males per 100 females)',
       'Educational services, and health care and social assistance',
       'relation_max_fee', 'White', 'ROOM Median rooms',
       'Hispanic or Latino (of any race)', 'Mean household income (dollars)',
       'VALUE Median (dollars)', 'Mean travel time to work (minutes)',
       'Total population', 'Rental vacancy rate', 'relation_min_fee',
       'HOUSING TENURE Renter-occupied',
       'Arts, entertainment, and recreation, and accommodation and food services',
       'SELECTED MONTHLY OWNER COSTS (SMOC) Housing units without a mortgage Median (dollars)',
       'SELECTED MONTHLY OWNER COSTS (SMOC) Housing units with a mortgage Median (dollars)',
       'Own children of the householder under 6 years', 'label', 'INTPTLONG',
       'INTPTLAT', 'geoid', 'Median age (years)', 'tract_id',

In [77]:
ids = combined_df['tract_id']
labels = combined_df['label']
lats = combined_df['INTPTLAT']
longs = combined_df['INTPTLONG']
econ_features = combined_df[['Own children of the householder 6 to 17 years',
       'YEAR OF ENTRY Foreign born', 'Median age (years)',
       'GROSS RENT Occupied units paying rent Median (dollars)',
       'Sex ratio (males per 100 females)',
       'Educational services, and health care and social assistance',
       'relation_max_fee', 'White', 'ROOM Median rooms',
       'Hispanic or Latino (of any race)', 'Mean household income (dollars)',
       'VALUE Median (dollars)', 'Mean travel time to work (minutes)',
       'Total population', 'Rental vacancy rate', 'relation_min_fee',
       'HOUSING TENURE Renter-occupied',
       'Arts, entertainment, and recreation, and accommodation and food services',
       'SELECTED MONTHLY OWNER COSTS (SMOC) Housing units without a mortgage Median (dollars)',
       'SELECTED MONTHLY OWNER COSTS (SMOC) Housing units with a mortgage Median (dollars)',
       'Own children of the householder under 6 years']]
image_features = combined_df[['house_num', 'shape_area']]
path_features = combined_df[['hospital_1', 'hospital_2', 'subway_1', 'subway_2', 'school_1', 'school_2']]

In [78]:
image_features.isnull().any()

house_num     False
shape_area    False
dtype: bool

In [79]:
la_lat_n = 34.746448
la_lat_s = 33.4
la_long_w = -118.861046
la_long_e = -117.015579

In [80]:
tracts = pd.DataFrame(list(zip(ids, lats, longs, labels)))
tracts.columns = ['id', 'lat', 'long', 'label']

In [81]:
tracts.describe()

,id,lat,long,label
count,1.629000e+03,1629.000000,1629.000000,1629.000000
mean,6.037409e+09,34.058677,-118.249140,0.165746
std,5.766448e+05,0.170100,0.196136,0.371967
min,6.037101e+09,33.708857,-118.817042,0.000000
25%,6.037208e+09,33.954281,-118.384482,0.000000
50%,6.037405e+09,34.052571,-118.270199,0.000000
75%,6.037544e+09,34.148759,-118.136930,0.000000
max,6.059002e+09,34.712733,-117.708673,1.000000


In [82]:
tracts.tail(10)

,id,lat,long,label
1619,6037920328,34.414692,-118.567931,0
1620,6037920329,34.410555,-118.550361,0
1621,6037920330,34.404404,-118.560631,0
1622,6037920331,34.397102,-118.545361,0
1623,6037920332,34.383179,-118.514000,0
1624,6037920334,34.387377,-118.567250,0
1625,6037920336,34.385597,-118.535884,0
1626,6037920337,34.373662,-118.512874,0
1627,6037920338,34.386084,-118.596430,0
1628,6037980015,33.773247,-118.288839,0


In [83]:
for index, row in tracts.iterrows():
    if (row['lat'] > la_lat_n or row['lat'] < la_lat_s or
            row['long'] > la_long_e or row['long'] < la_long_w):
        tracts = tracts.drop([index])

In [84]:
def get_distance(p1, p2):
    dist = math.sqrt((p1[0]-p2[0])**2 + (p1[1]-p2[1])**2)
    return dist

In [85]:
hospital_locations = pd.read_csv('./datasets/la_hospitals.csv')
hospital_x = hospital_locations

In [86]:
hospitals_x = hospital_locations['X']
hospitals_y = hospital_locations['Y']

hospitals = []
for n_long, n_lat in list(zip(hospitals_x, hospitals_y)):
    if (n_lat<=la_lat_n and n_lat>=la_lat_s and
            n_long<=la_long_e and n_long>=la_long_w):
        hospitals.append({"longitude": n_long, "latitude": n_lat, "label":0, "closest": -1})

In [87]:
for i, hospital in enumerate(hospitals):
    closest_distance = 1000000
    for index, row in tracts.iterrows():
        dist = get_distance((row['long'], row['lat']),
                            (hospital["longitude"], hospital["latitude"]))
        if dist < closest_distance:
            closest_distance = dist
            label = row['label']
            closest_i = index
    hospitals[i]["label"] = int(label)
    hospitals[i]["closest"] = int(closest_i)

In [88]:
subway_locations = pd.read_csv('./datasets/la_subway.csv')
subway_x = subway_locations['X']
subway_y = subway_locations['Y']

subway = []
for n_long, n_lat in list(zip(subway_x, subway_y)):
    if (n_lat<=la_lat_n and n_lat>=la_lat_s and
            n_long<=la_long_e and n_long>=la_long_w):
        subway.append({"longitude": n_long, "latitude": n_lat, "label":0, "closest": -1})

In [89]:
for i, station in enumerate(subway):
    closest_distance = 1000000
    for index, row in tracts.iterrows():
        dist = get_distance((row['long'], row['lat']),
                            (station["longitude"], station["latitude"]))
        if dist < closest_distance:
            closest_distance = dist
            label = row['label']
            closest_i = index
    subway[i]["label"] = int(label)
    subway[i]["closest"] = int(closest_i)

In [90]:
school_locations = pd.read_csv('./datasets/la_schools.csv')
schools_x = school_locations['X']
schools_y = school_locations['Y']

all_schools = []
for n_long, n_lat in list(zip(schools_x, schools_y)):
    if (n_lat<=la_lat_n and n_lat>=la_lat_s and
            n_long<=la_long_e and n_long>=la_long_w):
        all_schools.append({"longitude": n_long, "latitude": n_lat, "label":0, "closest": -1})

In [91]:
for i, school in enumerate(all_schools):
    closest_distance = 1000000
    for index, row in tracts.iterrows():
        dist = get_distance((row['long'], row['lat']),
                            (school["longitude"], school["latitude"]))
        if dist < closest_distance:
            closest_distance = dist
            label = row['label']
            closest_i = index
    all_schools[i]["label"] = int(label)
    all_schools[i]["closest"] = int(closest_i)

In [92]:
tract_lats = tracts['lat'].tolist()
tract_longs = tracts['long'].tolist()
tract_labels = tracts['label'].tolist()

In [93]:
tract_dicts = []
for i, vals in enumerate(list(zip(tract_longs, tract_lats, tract_labels))):
    t_dict = {"longitude": vals[0], "latitude": vals[1], "label":vals[2], "closest": i}
    tract_dicts.append(t_dict)

In [94]:
all_nodes = tract_dicts + hospitals + subway + all_schools

In [95]:
tract_indices = list(range(0, len(tracts)))
all_indices = list(range(0, len(all_nodes)))
hospital_indices = list(range(len(tracts), len(tracts) + len(hospitals)))
subway_indices = list(range(len(tracts) + len(hospitals), len(tracts) + len(hospitals) + len(subway)))
school_indices = list(range(len(tracts) + len(hospitals) + len(subway), len(tracts) + len(hospitals) + len(subway) + len(all_schools)))

In [96]:
len(hospitals), len(hospital_indices)

(164, 164)

**Creating the graph using NetworkX**

In [97]:
city_graph = nx.Graph()
color_map = []
# Create nodes
for i, node_dict in enumerate(all_nodes):
    city_graph.add_node(i,
                        pos=(node_dict["longitude"],
                             node_dict["latitude"])
                        )
    
    if node_dict["label"] == 1:
         color_map.append('purple')
    else:
        color_map.append('lightblue')

# Add hospital edges
for tract_i in all_indices:
    hospital_dict = {}
    for hospital_i in hospital_indices:
        hospital_dict[hospital_i] = get_distance((all_nodes[hospital_i]["longitude"], all_nodes[hospital_i]["latitude"]),
                                             (all_nodes[tract_i]["longitude"], all_nodes[tract_i]["latitude"]))
    
    closest2 = sorted(hospital_dict, key=hospital_dict.get, reverse=False)[0:2]
    
    for i in closest2:
        s_lat = all_nodes[i]["latitude"]
        s_long = all_nodes[i]["longitude"]
        distance = get_distance((all_nodes[tract_i]["longitude"], all_nodes[tract_i]["latitude"]),
                                (s_long, s_lat))
        city_graph.add_edge(tract_i, i, weight=distance)

# Add school edges
for tract_i in all_indices:
    school_dict = {}
    for school_i in school_indices:
        school_dict[school_i] = get_distance((all_nodes[school_i]["longitude"], all_nodes[school_i]["latitude"]),
                                             (all_nodes[tract_i]["longitude"], all_nodes[tract_i]["latitude"]))
    
    closest2 = sorted(school_dict, key=school_dict.get, reverse=False)[0:2]
    
    for i in closest2:
        s_lat = all_nodes[i]["latitude"]
        s_long = all_nodes[i]["longitude"]
        distance = get_distance((all_nodes[tract_i]["longitude"], all_nodes[tract_i]["latitude"]),
                                (s_long, s_lat))
        city_graph.add_edge(tract_i, i, weight=distance)

# Add subway edges
for tract_i in all_indices:
    subway_dict = {}
    for subway_i in subway_indices:
        subway_dict[subway_i] = get_distance((all_nodes[subway_i]["longitude"], all_nodes[subway_i]["latitude"]),
                                             (all_nodes[tract_i]["longitude"], all_nodes[tract_i]["latitude"]))
    
    closest2 = sorted(subway_dict, key=subway_dict.get, reverse=False)[0:2]
    
    for i in closest2:
        s_lat = all_nodes[i]["latitude"]
        s_long = all_nodes[i]["longitude"]
        distance = get_distance((all_nodes[tract_i]["longitude"], all_nodes[tract_i]["latitude"]),
                                (s_long, s_lat))
        city_graph.add_edge(tract_i, i, weight=distance)

In [98]:
# Create a dictionary of node attributes
attrs = {}
for i, node_dict in enumerate(all_nodes):
    if i in school_indices:
        node_type = [1, 0, 0, 0]
    elif i in hospital_indices:
        note_type = [0, 1, 0, 0]
    elif i in subway_indices:
        node_type = [0, 0, 1, 0]
    else:
        node_type = [0, 0, 0, 1]

    closest_id = node_dict["closest"]

    attrs[i] = {"econ_0": econ_features.iloc[closest_id, 0],
                "econ_1": econ_features.iloc[closest_id, 1],
                "econ_2": econ_features.iloc[closest_id, 2],
                "econ_3": econ_features.iloc[closest_id, 3],
                "econ_4": econ_features.iloc[closest_id, 4],
                "econ_5": econ_features.iloc[closest_id, 5],
                "econ_6": econ_features.iloc[closest_id, 6],
                "econ_7": econ_features.iloc[closest_id, 7],
                "econ_8": econ_features.iloc[closest_id, 8],
                "econ_9": econ_features.iloc[closest_id, 9],
                "econ_10": econ_features.iloc[closest_id, 10],
                "econ_11": econ_features.iloc[closest_id, 11],
                "econ_12": econ_features.iloc[closest_id, 12],
                "econ_13": econ_features.iloc[closest_id, 13],
                "econ_14": econ_features.iloc[closest_id, 14],
                "econ_15": econ_features.iloc[closest_id, 15],
                "econ_16": econ_features.iloc[closest_id, 16],
                "econ_17": econ_features.iloc[closest_id, 17],
                "econ_18": econ_features.iloc[closest_id, 18],
                "econ_19": econ_features.iloc[closest_id, 19],
                "econ_20": econ_features.iloc[closest_id, 20],
                "image_0": image_features["shape_area"][closest_id],
                "image_1": image_features['house_num'][closest_id],
                "path_0": path_features.iloc[closest_id, 0],
                "path_1": path_features.iloc[closest_id, 1],
                "path_2": path_features.iloc[closest_id, 2],
                "path_3": path_features.iloc[closest_id, 3],
                "path_4": path_features.iloc[closest_id, 4],
                "path_5": path_features.iloc[closest_id, 5],
                "latitude": lats[closest_id],
                "longitude": longs[closest_id],
                "node_type_s": node_type[0],
                "node_type_h": node_type[1],
                "node_type_c": node_type[2],
                "node_type_t": node_type[3]
                }


nx.set_node_attributes(city_graph, attrs)

In [99]:
pip install stellargraph

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [100]:
import stellargraph as sg
from stellargraph.mapper import GraphSAGENodeGenerator
from stellargraph.layer import GraphSAGE

from tensorflow.keras import layers, optimizers, losses, metrics, Model
import tensorflow as tf

from sklearn import preprocessing, feature_extraction, model_selection
from sklearn.calibration import calibration_curve
from sklearn.linear_model import LogisticRegressionCV
from sklearn.isotonic import IsotonicRegression

from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve

from stellargraph.calibration import TemperatureCalibration, IsotonicCalibration
from stellargraph.calibration import plot_reliability_diagram, expected_calibration_error

from stellargraph import datasets
from stellargraph import StellarGraph

In [101]:
attrs_df = pd.DataFrame(attrs).T
attrs_df

,econ_0,econ_1,econ_2,econ_3,econ_4,econ_5,econ_6,econ_7,econ_8,econ_9,...,path_2,path_3,path_4,path_5,latitude,longitude,node_type_s,node_type_h,node_type_c,node_type_t
0,489.0,1449.0,40.6,1609.0,95.8,0.25,2120.0,0.77,4.5,0.32,...,10.0,10.0,1.0,1.0,34.259474,-118.292987,0.0,0.0,0.0,1.0
1,351.0,846.0,52.3,2120.0,90.1,0.25,2016.0,0.88,5.4,0.04,...,11.0,11.0,1.0,1.0,34.267721,-118.290147,0.0,0.0,0.0,1.0
2,804.0,3024.0,36.6,1318.0,124.1,0.24,2120.0,0.79,3.6,0.43,...,10.0,10.0,1.0,1.0,34.252972,-118.290731,0.0,0.0,0.0,1.0
3,538.0,1686.0,39.7,1198.0,84.7,0.22,2120.0,0.71,4.0,0.38,...,9.0,9.0,1.0,1.0,34.251608,-118.281633,0.0,0.0,0.0,1.0
4,181.0,764.0,45.8,2111.0,84.5,0.26,2063.0,0.86,5.1,0.27,...,9.0,9.0,1.0,1.0,34.225079,-118.354188,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3096,250.0,706.0,38.3,1318.0,100.9,0.32,1744.0,0.53,5.3,0.53,...,2.0,5.0,1.0,1.0,34.121868,-118.173602,1.0,0.0,0.0,0.0
3097,429.0,1167.0,39.9,1171.0,99.0,0.20,1616.0,0.45,5.7,0.57,...,8.0,8.0,1.0,1.0,33.802651,-118.270199,1.0,0.0,0.0,0.0
3098,586.0,670.0,34.4,1538.0,117.0,0.13,1770.0,0.91,5.2,0.19,...,1.0,2.0,1.0,1.0,34.171214,-118.435606,1.0,0.0,0.0,0.0
3099,294.0,1530.0,45.3,1721.0,91.3,0.19,1788.0,0.26,4.7,0.14,...,4.0,6.0,1.0,1.0,34.058515,-118.326191,1.0,0.0,0.0,0.0


In [102]:
attrs_df.isnull().any()

econ_0         False
econ_1         False
econ_2         False
econ_3         False
econ_4         False
econ_5         False
econ_6         False
econ_7         False
econ_8         False
econ_9         False
econ_10        False
econ_11        False
econ_12        False
econ_13        False
econ_14        False
econ_15        False
econ_16        False
econ_17        False
econ_18        False
econ_19        False
econ_20        False
image_0        False
image_1        False
path_0         False
path_1         False
path_2         False
path_3         False
path_4         False
path_5         False
latitude       False
longitude      False
node_type_s    False
node_type_h    False
node_type_c    False
node_type_t    False
dtype: bool

In [103]:
scaler = StandardScaler()

In [104]:
# Feed the graph into GraphSAGE
sg_city_graph = StellarGraph.from_networkx(city_graph,
                                           node_features=pd.DataFrame(scaler.fit_transform(attrs_df)))

In [105]:
print(sg_city_graph.info())

StellarGraph: Undirected multigraph
 Nodes: 3101, Edges: 17756

 Node types:
  default: [3101]
    Features: float32 vector, length 35
    Edge types: default-default->default

 Edge types:
    default-default->default: [17756]
        Weights: range=[0, 0.561018], mean=0.0465423, std=0.0738956
        Features: none


In [106]:
labels = [int(node["label"]) for node in all_nodes]
y = np.zeros([len(labels), 2])

for i, val in enumerate(labels):
    if val == 0:
        y[i][0] = 1
        y[i][1] = 0
    elif val == 1:
        y[i][0] = 0
        y[i][1] = 1
    else:
        print("ERROR")

In [107]:
train_set, test_set = model_selection.train_test_split(
    pd.DataFrame(y), train_size=0.8, test_size=None, stratify=pd.DataFrame(y)
)
val_set = test_set

In [108]:
target_encoding = preprocessing.LabelBinarizer()

train_targets = target_encoding.fit_transform(train_set)
val_targets = target_encoding.fit_transform(val_set)
test_targets = target_encoding.transform(test_set)

In [109]:
batch_size = 150 # Try 200
dim_shape = [16, 4, 2] # Try 32 instead of 16
epochs = 400 # Try 500

In [110]:
for iteration in range(0, 10):
    train_set, test_set = model_selection.train_test_split(
        pd.DataFrame(y), train_size=0.8, test_size=None, stratify=pd.DataFrame(y)
    )
    val_set = test_set
    target_encoding = preprocessing.LabelBinarizer()
    train_targets = target_encoding.fit_transform(train_set)
    val_targets = target_encoding.fit_transform(val_set)
    test_targets = target_encoding.transform(test_set)

    generator = GraphSAGENodeGenerator(sg_city_graph, batch_size, dim_shape)
    train_gen = generator.flow(train_set.index, train_targets)

    graphsage_model = GraphSAGE(
        layer_sizes=dim_shape, generator=generator, bias=True
    )

    x_inp, x_out = graphsage_model.in_out_tensors()
    prediction = layers.Dense(units=train_targets.shape[1], activation="sigmoid")(x_out)

    model = Model(inputs=x_inp, outputs=prediction)
    model.compile(
        optimizer=optimizers.Adam(learning_rate=0.01),
        loss=losses.binary_crossentropy,
        metrics=["acc"],
    )

    val_gen = generator.flow(val_set.index, val_targets)
    test_gen = generator.flow(test_set.index, test_targets)

    history = model.fit(
        train_gen, epochs=epochs, validation_data=val_gen, shuffle=True,
    )

    all_gen = generator.flow(list(city_graph.nodes), y)
    all_predictions = model.predict(all_gen).squeeze()[..., np.newaxis]
    all_predictions_df = pd.DataFrame(all_predictions[:,0], index=list(list(city_graph.nodes)))
    test_preds = all_predictions_df.loc[test_set.index, :]

    test_predictions = test_preds.values
    test_predictions_class = ((test_predictions > 0.5) * 1).flatten()
    test_df = pd.DataFrame(
        {
            "Predicted_score": test_predictions.flatten(),
            "Predicted_class": test_predictions_class,
            "True": test_targets[:, 0],
        }
    )
    
    if iteration == 0:
        all_vals_df = pd.DataFrame(
            {
                "Predicted_score": test_predictions.flatten(),
                "Predicted_class": test_predictions_class,
                "True": test_targets[:, 0],
            }
        )
    else:
        all_vals_df = pd.concat([all_vals_df, test_df], ignore_index=True, sort=False)

    print(iteration)

Epoch 1/400
17/17 [==============================] - 15s 554ms/step - loss: 0.7125 - acc: 0.6520 - val_loss: 0.6579 - val_acc: 0.7987
Epoch 2/400
17/17 [==============================] - 8s 492ms/step - loss: 0.6247 - acc: 0.7980 - val_loss: 0.5772 - val_acc: 0.7987
Epoch 3/400
13/17 [=====================>........] - ETA: 1s - loss: 0.5601 - acc: 0.7910

KeyboardInterrupt: ignored

In [ ]:
all_vals_df

In [ ]:
all_vals_df.to_csv("./datasets/la_graphsage.csv", encoding='utf-8', index=False)